# Практические задания 19
* Лекция 37 - Работа с базами данных из интерфейса Python


#### Онлайн-библиотека
Вы создаёте пошагово консольное приложение на Python, работающее с базой данных MySQL и MongoDB. Ваша цель – проанализировать работу интернет-библиотеки. Библиотека работает следующим образом: пользователь выбирает книгу и оплачивает возможность ее чтения в приложении магазина до конца календарного года. 1 января каждого года доступ всех пользователей ко всем книгам закрывается. Вы получили для анализа три csv-файла: об имеющихся в магазине книгах и стоимости их чтения, о пользователях библиотеки и о выданных правах на чтение.   
Проект включает в себя следующие этапы:
1. Создание базы данных и таблиц
2. Загрузка данных из файлов в созданную базу данных
3. Написание запросов к базе данных
4. Создание меню для поиска информации по книгам и пользователям
5. Фиксация поисков в MongoDB
6. Анализ самых популярных запросов


### Онлайн-библиотека. Часть 1


1. **Создание базы данных библиотеки**  
Создайте новую базу данных MySQL под названием library_<номер группы>. (Добавьте так же свое имя, если работаете индивидуально)
После этого выполните проверку: если база данных успешно создана (или уже существует), выведите сообщение:   
`Database 'library_...' created or already exists.`


In [23]:
import pymysql

config = {
    'host': 'ich-edit.edu.itcareerhub.de',
    'user': 'ich1',
    'password': 'ich1_password_ilovedbs',
}

db_name = "library_test"

with pymysql.connect(**config) as connection:
    with connection.cursor() as cursor:
        cursor.execute(f"CREATE DATABASE IF NOT EXISTS {db_name}")
        cursor.execute("SHOW DATABASES")
        databases = [row[0] for row in cursor]

        if db_name in databases:
            print(f"Database '{db_name}' created or already exists.")
        else:
            print("Something went wrong. Database not found.")

Database 'library_test' created or already exists.


In [24]:
databases

['00_',
 '010825_AA',
 '010825_AL',
 '010825_ARSENII',
 '010825_Denis',
 '010825_Illia',
 '010825_KV',
 '010825_LK',
 '010825_LKT',
 '010825_LSV',
 '010825_SRV',
 '010825_STRK',
 '010825_Vladimir156651',
 '010825_abalmas',
 '010825_gaskarov',
 '010825_kateryna',
 '010825_ptm_Dmytro',
 '01112024_dam_DilNatalia',
 '020625_Davydchenko',
 '020625_IRYNAR',
 '020625_KLEIN',
 '020625_KUIN',
 '020625_MAKSAKOVAANNA',
 '020625_NAUMOVANTON',
 '020625_POMIATIKHINAMARIIA',
 '020625_SURZHENKO',
 '020625_Srapian_Lamara',
 '020625_alisiia_bisilova',
 '020625_churish',
 '030325_AKIR',
 '030325_BAR',
 '030325_BOM',
 '030325_Churish',
 '030325_Dmitriy',
 '030325_InaRus',
 '030325_Iryna',
 '030325_KhaIR',
 '030325_Liliya',
 '030325_MOR',
 '030325_MyClass',
 '030325_STR',
 '030325_Vesna',
 '030325_dam_ClassWork',
 '03072025_Prokofeva',
 '041124_FEDIK',
 '041124_Petrishchev',
 '041124_Ukhach',
 '041124_Yasynetska',
 '041124_Zolotukhin',
 '050824_AW',
 '06022024_Max',
 '06122024_Max_Bike',
 '06122024_Max_Tax

2. **Создание таблиц для данных библиотеки**  
Расширьте предыдущую программу: после создания базы данных library, создайте в ней три таблицы:  
* books — для хранения информации о книгах;
* users — для информации о пользователях
* reading — для информации о выданных правах на чтение.
    
Код для создания таблиц: 
```
books_query = """
   CREATE TABLE IF NOT EXISTS books (
       id INT PRIMARY KEY,
       title VARCHAR(200),
       author VARCHAR(100),
       price DECIMAL(10, 2)
   )
"""

users_query = """
   CREATE TABLE IF NOT EXISTS users (
       id INT PRIMARY KEY,
       username VARCHAR(100),
       firstname VARCHAR(100),
       lastname VARCHAR(100),
       country VARCHAR(100)
   )
"""

reading_query = """
   CREATE TABLE IF NOT EXISTS reading (
       id INT AUTO_INCREMENT PRIMARY KEY,
       userid INT,
       bookid INT,
       readingyear INT
   )
"""
```
После создания таблиц выведите список всех таблиц в текущей базе.
Пример вывода:   
```
Tables in 'library_...':
- books
- users
- reading
```

In [25]:
books_query = """
   CREATE TABLE IF NOT EXISTS books (
       id INT PRIMARY KEY,
       title VARCHAR(200),
       author VARCHAR(100),
       price DECIMAL(10, 2)
   )
"""

users_query = """
   CREATE TABLE IF NOT EXISTS users (
       id INT PRIMARY KEY,
       username VARCHAR(100),
       firstname VARCHAR(100),
       lastname VARCHAR(100),
       country VARCHAR(100)
   )
"""

reading_query = """
   CREATE TABLE IF NOT EXISTS reading (
       id INT AUTO_INCREMENT PRIMARY KEY,
       userid INT,
       bookid INT,
       readingyear INT
   )
"""

with pymysql.connect(**config) as connection:
    with connection.cursor() as cursor:
        # Шаг 2 — создать таблицы
        cursor.execute(f"USE {db_name}")

        cursor.execute(books_query)
        cursor.execute(users_query)
        cursor.execute(reading_query)

        # Вывод списка таблиц
        cursor.execute("SHOW TABLES")
        tables = [row[0] for row in cursor]

        print(f"Tables in '{db_name}':")
        for table in tables:
            print(f"- {table}")


Tables in 'library_test':
- books
- reading
- users


3. **Загрузка данных из файлов**  
Прочитайте файлы books.csv, users.csv, readingyear.csv. Перенесите данные из этих файлов в созданные таблицы БД. В случае успешного завершения операции программа должна вывести сообщение о количестве перенесенных записей.
Пример файла book.csv:   
ID,Title,Author,Price  
3823,The Code Breaker,Walter Isaacson,0.03  
8561,The Secret History,Donna Tartt,0.36  
Пример вывода:   
2 books loaded.


In [26]:
import csv

# Функция для чтения данных из CSV-файла
def read_csv_file(file_path):
    data = []
    with open(file_path, mode='r') as file:
        csv_reader = csv.reader(file)
        next(csv_reader)  # Пропускаем заголовок
        for row in csv_reader:
            data.append(tuple(row))
    return data

# Чтение данных из CSV-файлов
books_data = read_csv_file('books.csv')
users_data = read_csv_file('users.csv')
reading_data = read_csv_file('readingyear.csv')

# Подключение к серверу MySQL
with pymysql.connect(**config) as connection:
    with connection.cursor() as cursor:
        cursor.execute(f"USE {db_name}")
        # Вставка данных в таблицу books
        books_query = "INSERT INTO books (id, title, author, price) VALUES (%s, %s, %s, %s)"
        cursor.executemany(books_query, books_data)

        # Вставка данных в таблицу users
        users_query = "INSERT INTO users (id, username, firstname, lastname, country) VALUES (%s, %s, %s, %s, %s)"
        cursor.executemany(users_query, users_data)

        # Вставка данных в таблицу reading
        reading_query = "INSERT INTO reading (userid, bookid, readingyear) VALUES (%s, %s, %s)"
        cursor.executemany(reading_query, reading_data)

        # Подтверждение изменений
        connection.commit()

        # Вывод количества перенесенных записей
        print(f"{len(books_data)} books loaded.")
        print(f"{len(users_data)} users loaded.")
        print(f"{len(reading_data)} reading records loaded.")


55 books loaded.
50 users loaded.
149 reading records loaded.


In [27]:
reading_data

[('8233', '7492', '2025'),
 ('5155', '8300', '2021'),
 ('3181', '221', '2023'),
 ('4969', '2064', '2024'),
 ('6582', '4647', '2022'),
 ('2770', '953', '2024'),
 ('3982', '7808', '2024'),
 ('6582', '5120', '2021'),
 ('479', '1821', '2021'),
 ('2826', '7864', '2023'),
 ('1944', '4676', '2020'),
 ('7889', '4022', '2022'),
 ('4122', '4134', '2023'),
 ('2629', '7011', '2024'),
 ('9395', '2064', '2025'),
 ('67', '9829', '2025'),
 ('5892', '7011', '2022'),
 ('6661', '4022', '2021'),
 ('2770', '1008', '2024'),
 ('1810', '6920', '2022'),
 ('515', '7001', '2021'),
 ('1944', '8903', '2022'),
 ('7821', '1821', '2025'),
 ('6661', '6', '2025'),
 ('2629', '9376', '2024'),
 ('515', '4134', '2024'),
 ('5155', '6', '2020'),
 ('5892', '1008', '2021'),
 ('893', '9474', '2022'),
 ('7889', '221', '2021'),
 ('2006', '5174', '2024'),
 ('4122', '3823', '2024'),
 ('515', '7864', '2020'),
 ('4969', '167', '2024'),
 ('207', '9697', '2022'),
 ('4307', '7011', '2025'),
 ('6864', '9849', '2023'),
 ('6753', '7492', '

4. **Проверка загрузки данных**  
Подключитесь к созданной и заполненной базе данных. Выполните следующие запросы и выведите результаты в консоль:  
* Количество записей в таблице books.
* Первые 10 записей в таблице users.
* Записи таблицы readingyear за 2025 год.
  
Пример вывода: 
Количество записей в таблице books: 55

Первые 10 записей в таблице users:
```
(67, 'jone', 'Anna', 'Jones', 'Dresden')
(207, 'da', 'David', 'Doe', 'Tokyo')
(431, 'jamesdo', 'James', 'Doe', 'Sydney')
...
```
Записи таблицы reading за 2025 год:
```
(51, 8233, 7492, 2025)
(65, 9395, 2064, 2025)
(66, 67, 9829, 2025)
...
```

In [28]:
with pymysql.connect(**config) as connection:
    with connection.cursor() as cursor:
        cursor.execute(f"USE {db_name}")
        # Количество записей в таблице books
        cursor.execute("SELECT COUNT(*) FROM books")
        books_count = next(cursor)[0]
        print(f"Количество записей в таблице books: {books_count}")

        # Первые 10 записей в таблице users
        cursor.execute("SELECT * FROM users LIMIT 10")
        print("\nПервые 10 записей в таблице users:")
        for record in cursor:
            print(record)

        # Записи таблицы reading за 2025 год
        cursor.execute("SELECT * FROM reading WHERE readingyear = 2025")
        print("\nЗаписи таблицы reading за 2025 год:")
        for record in cursor:
            print(record)



Количество записей в таблице books: 55

Первые 10 записей в таблице users:
(67, 'jone', 'Anna', 'Jones', 'Dresden')
(207, 'da', 'David', 'Doe', 'Tokyo')
(431, 'jamesdo', 'James', 'Doe', 'Sydney')
(479, 'dawilson', 'Dan', 'Wilson', 'London')
(515, 'jjon', 'James', 'Jones', 'Boston')
(527, 'smi', 'Olha', 'Smith', 'Boston')
(843, 'jjjj', 'James', 'Johnson', 'Berlin')
(893, 'maryw', 'Mary', 'Wilson', 'Dresden')
(1152, 'dajone', 'David', 'Jones', 'Berlin')
(1227, 'sopwi', 'Sophia', 'Williams', 'Tokyo')

Записи таблицы reading за 2025 год:
(1, 8233, 7492, 2025)
(15, 9395, 2064, 2025)
(16, 67, 9829, 2025)
(23, 7821, 1821, 2025)
(24, 6661, 6, 2025)
(36, 4307, 7011, 2025)
(46, 4122, 9829, 2025)
(50, 5155, 4850, 2025)
(70, 6410, 5174, 2025)
(77, 2826, 6920, 2025)
(78, 6582, 4134, 2025)
(81, 8233, 1117, 2025)
(98, 8665, 8300, 2025)
(105, 8233, 9474, 2025)
(106, 4975, 7011, 2025)
(107, 4969, 9829, 2025)
(114, 4975, 6716, 2025)
(121, 7889, 1905, 2025)
(124, 6864, 5120, 2025)
(132, 7821, 4676, 2025)

5. **Поиск авторов**   
Напишите функцию, которая запрашивает у пользователя имя автора (можно ввести полное имя или его часть) и выводит все записи из таблицы books, где есть совпадение с вводом пользователя.
Пример ввода:   
Введите имя автора (или его часть):  leo  

Пример вывода:   
Подходящие книги:  
ID: 1905, Title: Anna Karenina, Author: Leo Tolstoy, Price: 0.32  
ID: 9829, Title: War and Peace, Author: Leo Tolstoy, Price: 0.59


In [29]:
def search_books_by_author(author_name):
    # Подключение к серверу MySQL
    with pymysql.connect(**config) as connection:
        with connection.cursor() as cursor:
            cursor.execute(f"USE {db_name}")
            # Запрос имени автора у пользователя
            #author_name = input("Введите имя автора (или его часть): ")

            # SQL-запрос для поиска книг по имени автора
            query = """
                SELECT id, title, author, price
                FROM books
                WHERE author LIKE %s
            """
            cursor.execute(query, ('%' + author_name + '%',)) # LIKE '%leo%'

            # Вывод результатов
            count = 0
            print("\nПодходящие книги:")
            for row in cursor:
                print(f"ID: {row[0]}, Title: {row[1]}, Author: {row[2]}, Price: {row[3]}")
                count += 1
            if count == 0:
                print("не найдены.")

# Вызов функции
author_name = input("Введите имя автора (или его часть): ")
search_books_by_author(author_name)



Введите имя автора (или его часть):  leo



Подходящие книги:
ID: 1905, Title: Anna Karenina, Author: Leo Tolstoy, Price: 0.32
ID: 9829, Title: War and Peace, Author: Leo Tolstoy, Price: 0.59


6. **Поиск пользователей и книг, которые они читали**   
Напишите функцию, которая запрашивает у пользователя имя клиента библиотеки (можно ввести полное имя или фамилию или их часть) и выводит все записи из базы данных о книгах, прочитанных подходящими клиентами.
Пример ввода:   
Введите имя клиента (или его часть):  jon  
Пример вывода:   
Клиенты и прочитанные ими книги:  

Клиент: Anna Jones  
ID: 9829, Title: War and Peace, Author: Leo Tolstoy, Year: 2025  

Клиент: James Jones  
ID: 7001, Title: The Kite Runner, Author: Khaled Hosseini, Year: 2021  
ID: 4134, Title: The Canterbury Tales, Author: Geoffrey Chaucer, Year: 2024  
ID: 7864, Title: Educated, Author: Tara Westover, Year: 2020  

Клиент: David Jones  
ID: 4676, Title: The Plot, Author: Jean Hanff Korelitz, Year: 2021  
ID: 4647, Title: The Girl on the Train, Author: Paula Hawkins, Year: 2020  

Клиент: Mary Jones  
не читал книг.


In [32]:
def search_books_by_client(client_name):
    # Подключение к серверу MySQL
    with pymysql.connect(**config) as connection:
        with connection.cursor() as cursor:
            cursor.execute(f"USE {db_name}")
            # Запрос имени клиента у пользователя
            #client_name = input("Введите имя клиента (или его часть): ")

            # SQL-запрос для поиска клиентов по имени
            query = """
                SELECT id, firstname, lastname
                FROM users
                WHERE firstname LIKE %s OR lastname LIKE %s
            """
            cursor.execute(query, ('%' + client_name + '%', '%' + client_name + '%'))
            user_list = [user for user in cursor]

            print("\nКлиенты и прочитанные ими книги:")
            for user in user_list:
                user_id = user[0]
                firstname = user[1]
                lastname = user[2]

                # SQL-запрос для поиска книг, прочитанных клиентом
                books_query = """
                    SELECT b.id, b.title, b.author, r.readingyear
                    FROM reading r
                    JOIN books b ON r.bookid = b.id
                    WHERE r.userid = %s
                """
                cursor.execute(books_query, (user_id,))

                count_books = 0
                print(f"\nКлиент: {firstname} {lastname}")
                for book in cursor:
                    count_books += 1
                    print(f"ID: {book[0]}, Title: {book[1]}, Author: {book[2]}, Year: {book[3]}")
                if count_books == 0:
                    print("не читал книг.")
            if len(user_list) == 0:
                print("не найдены.")

# Вызов функции
search_books_by_client()


Введите имя клиента (или его часть):  anna



Клиенты и прочитанные ими книги:

Клиент: Anna Jones
ID: 9829, Title: War and Peace, Author: Leo Tolstoy, Year: 2025

Клиент: Anna Brown
не читал книг.

Клиент: Anna Williams
ID: 1821, Title: The Four Winds, Author: Kristin Hannah, Year: 2025
ID: 6484, Title: The Great Gatsby, Author: F. Scott Fitzgerald, Year: 2022
ID: 8007, Title: The Catcher in the Rye, Author: J.D. Salinger, Year: 2020
ID: 8926, Title: The Sun Also Rises, Author: Ernest Hemingway, Year: 2020
ID: 4676, Title: The Plot, Author: Jean Hanff Korelitz, Year: 2025
ID: 9474, Title: The Bone Shard Daughter, Author: Andrea Stewart, Year: 2024


In [36]:
user_id = 8233

with pymysql.connect(**config) as connection:
    with connection.cursor() as cursor:
        cursor.execute(f"USE {db_name}")
        books_query = """
                    SELECT b.id, b.title, b.author, r.readingyear
                    FROM reading r
                    JOIN books b ON r.bookid = b.id
                    WHERE r.userid = %s
                """
        cursor.execute(books_query, (user_id,))
        print(next(cursor))
        print(next(cursor))

(7492, "The Witch's Heart", 'Genevieve Gornichec', 2025)
(5174, 'Crime and Punishment', 'Fyodor Dostoevsky', 2022)
